# ***Proyecto EDA: Airbnb*** 
## ***Limpieza y Transformación***

**Autor:** Óscar Fernández-Chinchilla López

![Logo Airbnb](../images/airbnb-logo.png)

# 📋 Índice

1. [Introducción y objetivos de la limpieza](#introduccion)
2. [Carga del dataset original e importación de las librerias](#carga)
3. [Limpieza de columnas](#limpieza_col)
3. [Tratamiento de valores nulos](#nulos)
   - [Visualización de nulos](#vis-nulos)
   - [Imputación / Eliminación](#imputacion)
4. [Corrección de tipos de datos](#tipos)
5. [Estandarización de variables categóricas](#categoricas)
6. [Corrección de incoherencias en los datos](#incoherencias)
7. [Detección y tratamiento de outliers](#outliers)
8. [Comprobación final del dataset](#comprobacion)
9. [Guardado del dataset limpio](#guardado)

<a id="introduccion"></a>
## 1️⃣ Introducción y objetivos de la limpieza <a id="introduccion"></a>


Para poder realizar un análisis fiable del dataset y extraer conclusiones realmente
representativas, es necesario corregir los problemas identificados durante la fase de
exploración. Por ello, en este notebook vamos a llevar a cabo un proceso de limpieza y
transformación del dataset.

## 🎯 Objetivos de la limpieza:

- **Renombrar y limpiar** los nombres de las columnas para facilitar su manejo.
- **Eliminar** duplicados que puedan generar ruido o sesgos en el análisis.
- **Tratar** los valores nulos, tomando decisiones justificadas en cada caso:
  - **Eliminar** columnas o filas con demasiados nulos o información irrelevante
  - **Imputar** valores cuando la información sea importante mantenerla
- **Corregir** los tipos de datos que no sean coherentes (ej.: `price` como `object` → `float`)
- **Estandarizar** categorías con errores ortográficos o incoherencias (ej.: boroughs mal escritos)
- **Detectar y ajustar outliers** para evitar que afecten de forma negativa a análisis posteriores
- **Formatear** las fechas correctamente para futuros cálculos temporales

---

Una vez completado este proceso, obtendremos un dataset **limpio y preparado** para continuar con
el análisis y la visualización avanzada.



# 2️⃣ Carga del dataset original e importación de librerías <a id="carga"></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Cargamos el dataset original
airbnb_df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False)

# Creamos una copia para trabajar con ella y mantener el original intacto en caso de error
airbnb_clean = airbnb_df.copy()

In [7]:
airbnb_clean.head(2)

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN


In [10]:
airbnb_clean.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license'],
      dtype='object')

In [11]:
airbnb_clean["instant_bookable"].unique()

array([False, True, nan], dtype=object)

<a id="normalización"></a>
# 3️⃣ Normalización de las columnas

In [ ]:
# Aqui creamos una función destinada a normalizar los nombres de las columnas para que sean más legibles.
def normalize_column_names(df):
    df_cleaned = df.copy()
    df_cleaned.columns = (
        df_cleaned.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
    return df_cleaned

In [ ]:
# Una vez tenemos la función creada, la utilizamos con la copia del dataset original para trabajar
airbnb_clean = normalize_column_names(airbnb_clean)
airbnb_clean.columns

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')

#### Como se puede observar en el anterior output, todos los nombres de las columnas del dataset aparecen en minúscula, sin espacios en los extremos y con separadores de "_" entre palabras

<a id="normalización"></a>
# 4. Normalización de las columnas

In [6]:

#Aqui definiremos una función para limpiar las columnas monetarias del dataset, las cuales son pasadas por parámetro junto al dataframe
def clean_monetary_columns(df,monetary_columns):
    # Realizamos una copia de dataframe
    df_cleaned = df.copy()
    
    # Bucle que itera en la lista pasa de columnas monetarias
    for col in monetary_columns:
        # Comprobamos que la columna existe en el dataframe, y en el caso de que exista realizamos las siguientes operaciones
        if col in df_cleaned.columns:
            df_cleaned[col] = (
                df_cleaned[col].astype(str)       # Convierto el objeto a String para poder operar
                .str.replace(r'[^0-9\.\-]', '', regex=True)         # Elimino los simbolos cambiandolos por un espacio en blanco
                .str.strip()        # Los elimino de los laterales
            )
            # Convertimos las columnas a tipo numérico
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce")
    return df_cleaned

In [9]:
monetary_columns = ["price", "service_fee"]
airbnb_clean = clean_monetary_columns(airbnb_clean,monetary_columns)

In [12]:
airbnb_clean.price.info

<bound method Series.info of 0          966.0
1          142.0
2          620.0
3          368.0
4          204.0
           ...  
102594     844.0
102595     837.0
102596     988.0
102597     546.0
102598    1032.0
Name: price, Length: 102599, dtype: float64>

In [13]:
airbnb_clean.service_fee.info

<bound method Series.info of 0         193.0
1          28.0
2         124.0
3          74.0
4          41.0
          ...  
102594    169.0
102595    167.0
102596    198.0
102597    109.0
102598    206.0
Name: service_fee, Length: 102599, dtype: float64>

In [ ]:
def clean_categorical_values(df):
    df_cleaned = df.copy()
    # Ahora limpiaremos y normalizaremos los valores categóricos de la columna neighbourhood_group
    if "neighbourhood_group" in df_cleaned.columns:
        df_cleaned["neighbourhood_group"] =  df_cleaned["neighbourhood_group"].replace(
            {
                "brookln" : "Brooklyn",
                "manhatan": "Manhattan"
            }
        )
        # Reemplazar valores nulos por "Unknown"
        df_cleaned["neighbourhood_group"] = df_cleaned["neighbourhood_group"].fillna("Unknown")
    
    # Inputamos los nulos de la columna "host_identity_verified"
    if "host_identity_verified" in df_cleaned.columns:
        df_cleaned["host_identity_verified"] = df_cleaned["host_identity_verified"].fillna("Unknown")
    
    # Inputamos también los nombres de la columna "neighbourhood"
    if "neighbourhood" in df_cleaned.columns:
        df_cleaned["neighbourhood"] = df_cleaned["neighbourhood"].split(",",n=1).str[0].str.strip()
    
    # Convertimos la columna "instant_bookable" a Booleano 
    if "instant_bookable" in df_cleaned.columns:
            df_cleaned["instant_bookable"] =  df_cleaned["instant_bookable"].replace(
            {
                "False" : False,
                "True": True
            }
            )
    df_cleaned["instant_bookable"] = df_cleaned["instant_bookable"].astype("boolean")
    return df_cleaned
    

In [ ]:
def format_dates(df):
    df_cleaned = df.copy()
    # Lo siguiente que realizaremos será formatear las fechas de la columna "last review"
    if "last_review" in df_cleaned.columns:
        df_cleaned['last_review'] = pd.to_datetime(df_cleaned['last_review'], errors='coerce')
    return df_cleaned

In [ ]:
def convert_numeric_types(df, list_types):
    """
    Convierte a numérico las columnas indicadas en list_types.
    Usa Int64 si hay NaN para mantener consistencia.
    """
    df_cleaned = df.copy()

    for col in list_types:
        if col in df_cleaned.columns:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce").astype("Int64")

    return df_cleaned